In [81]:
import keras
import pandas as pd
from keras.datasets import fashion_mnist 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.optimizers import RMSprop 
from keras.optimizers import SGD

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


try:
    df = pd.read_csv("data/ecoli.csv", header=None, delimiter=";")
except:
  print("No data found")

# Initializing data:
num_classes = len(df.loc[:,8].unique())

df = df.dropna(axis=0, how='any')
df_ind = df.loc[:,1:7]
df_dep = df.loc[:,8]

# Should I do Onehot encoding here?
labelencoder= LabelEncoder()
df_dep= labelencoder.fit_transform(df_dep)

X_train, X_test, Y_train, Y_test = train_test_split( df_ind , df_dep , test_size = 0.2, random_state = 0)

# convert class vectors to binary class matrices
# What is done here? Is this like one-hot encoding?
Y_train = keras.utils.to_categorical(Y_train, num_classes) 
Y_test = keras.utils.to_categorical(Y_test, num_classes)


def train_model(model, batch_size, epochs):
    history = model.fit(X_train, Y_train,
        batch_size=batch_size, epochs=epochs,
        verbose=0, validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose=0) 
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return score

def build_model(number_of_neurons,number_of_layers,batch_size, epochs,opt):
    model = Sequential()
    model.add(Dense(number_of_neurons, activation='relu', input_dim = 7))
    
    # Adding layers
    for i in range(number_of_layers-1):
        model.add(Dense(number_of_neurons, activation='relu'))
    
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 
    return train_model(model,batch_size, epochs)

def main():
    epochs = 80
    result_list = []

    testing_parameters = [[100,1,20,epochs,'RMSprop'],[100,2,20,epochs,'RMSprop'],[100,3,20,epochs,'RMSprop'],
                         [100,1,20,epochs,'sgd'],[100,2,20,epochs,'sgd'],[100,3,20,epochs,'sgd']]
    
    # Running the mlp_with_no_keras model
    
    for x in testing_parameters:
        acc = build_model(x[0],x[1],x[2],x[3],x[4])
        x.append(acc[0])
        x.append(acc[1])
        result_list.append(x)
    
    # Create the pandas DataFrame 
    df = pd.DataFrame(result_list, columns = ['number_of_neurons', 'number_of_layers', 'batch_size', 'epochs',
                                             'optimizer','loss', 'accuracy']) 
    print(df.to_string())
        


if __name__ == "__main__":
    main()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 100)               800       
_________________________________________________________________
dense_124 (Dense)            (None, 8)                 808       
Total params: 1,608
Trainable params: 1,608
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.706685087260078
Test accuracy: 0.8235294222831726
Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_125 (Dense)            (None, 100)               800       
_________________________________________________________________
dense_126 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_127 (Dense)            (N

,0
0,AAT_ECOLI;0.49;0.29;0.48;0.50;0.56;0.24;0.35;cp
1,ACEA_ECOLI;0.07;0.40;0.48;0.50;0.54;0.35;0.44;cp
2,ACEK_ECOLI;0.56;0.40;0.48;0.50;0.49;0.37;0.46;cp
3,ACKA_ECOLI;0.59;0.49;0.48;0.50;0.52;0.45;0.36;cp
4,ADI_ECOLI;0.23;0.32;0.48;0.50;0.55;0.25;0.35;cp
...,...
331,TREA_ECOLI;0.74;0.56;0.48;0.50;0.47;0.68;0.30;pp
332,UGPB_ECOLI;0.71;0.57;0.48;0.50;0.48;0.35;0.32;pp
333,USHA_ECOLI;0.61;0.60;0.48;0.50;0.44;0.39;0.38;pp
334,XYLF_ECOLI;0.59;0.61;0.48;0.50;0.42;0.42;0.37;pp
